## Instalação de Requisitos

In [ ]:
# Lendo o arquivo requisitos.txt
with open('requisitos.txt', 'r') as file:
    requirements = file.readlines()

# Removendo espaços em branco e quebras de linha
requirements = [requirement.strip() for requirement in requirements]

# Instalando as dependências usando pip
!pip3 install -U { ' '.join(requirements) }

## Importando bibliotecas

In [ ]:
import pandas as pd # type: ignore
import numpy as np # type: ignore

## Importando o DataSet

In [ ]:
file_path = 'NHANES_age_prediction.csv'
data = pd.read_csv(file_path, header=None)
data.head()

## Explorando o DATASET
1. Dimensões do dataset
2. Quantidade de valores NULOS
3. Valores UNICOS
4. Estatísticas

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data[9].value_counts()

In [ ]:
data.describe()

## Preparando o Dataset